# Config Tool

The `uwtools` API's `config` module provides functions to create and manipulate configuration files, objects, and dictionaries.

For more information, please see the <a href="https://uwtools.readthedocs.io/en/stable/sections/user_guide/api/config.html">uwtools.api.config</a> Read the Docs page.

## Table of Contents

* [Getting Config Objects](#Getting-Config-Objects)
* [Config Depth Limitations](#Config-Depth-Limitations)
* [Realizing Configs](#Realizing-Configs)
    * [Updating Configs](#Updating-Configs)
    * [Using the `key_path` Parameter](#Using-the-key_path-Parameter)
    * [Using the `values_needed` Parameter](#Using-the-values_needed-Parameter)
    * [Using the `total` Parameter](#Using-the-total-Parameter)
* [Realizing Configs to a Dictionary](#Realizing-Configs-to-a-Dictionary)
<!--cell 0-->

In [1]:
from pathlib import Path
from uwtools.api import config
from uwtools.api.logging import use_uwtools_logger

use_uwtools_logger()

## Getting Config Objects

The `config` tool can create configuration objects given a Python ``dict`` or a file in one of five different formats: FieldTable, INI, Fortran namelist, Shell, or YAML. `config.get_yaml_config` is demonstrated here, but the config module also has similar functions for each of the other supported formats: `get_fieldtable_config()`,  `get_ini_config()`, `get_nml_config()`, and `get_sh_config()`.
<!--cell 2-->

In [2]:
help(config.get_yaml_config)

Help on function get_yaml_config in module uwtools.api.config:

get_yaml_config(config: Union[dict, str, pathlib.Path, NoneType] = None, stdin_ok: bool = False) -> uwtools.config.formats.yaml.YAMLConfig
    Get a ``YAMLConfig`` object.

    :param config: YAML file or ``dict`` (``None`` => read ``stdin``).
    :param stdin_ok: OK to read from ``stdin``?
    :return: An initialized ``YAMLConfig`` object.



The `stdin_ok` argument can be used to permit reads from `stdin`, but this is a rare use case beyond the scope of this notebook that will not be discussed here.

`get_yaml_config()` can take input from a Python `dict` or a YAML file like the one below.
<!--cell 4-->

In [3]:
%%bash
cat fixtures/config/get-config.yaml

greeting: Hello
recipient: World


Paths to config files can be provided either as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. Since `get_yaml_config()` is used here, a `YAMLConfig` object is returned.
<!--cell 6-->

In [4]:
config1 = config.get_yaml_config(
    config=Path("fixtures/config/get-config.yaml")
)
print(type(config1))
print(config1)

<class 'uwtools.config.formats.yaml.YAMLConfig'>
greeting: Hello
recipient: World


Providing a Python `dict` will create a UW `Config` object with format matching the function used.
<!--cell 8-->

In [5]:
input_config = {"message": {"greeting":"Hi", "recipient":"Earth"}}
config2 = config.get_yaml_config(
    config=input_config
)
print(config2)

message:
  greeting: Hi
  recipient: Earth


## Config Depth Limitations

Some config formats have limitations on the depth of their nested configs. Shell configs, for example, may only contain top-level, bash-syntax `key=value` pairs.
<!--cell 10-->

In [6]:
config.get_sh_config(
    config={"greeting":"Salutations", "recipient":"Mars"}
)

greeting=Salutations
recipient=Mars

Shell configs cannot be nested, and any attempt to do so will raise a `UWConfigError`.
<!--cell 12-->

In [30]:
try: 
    config.get_sh_config(
        config={"message": {"greeting":"Salutations", "recipient":"Mars"}}
    )
except Exception as e:
    print(e)

Cannot instantiate depth-1 SHConfig with depth-2 config


When creating INI configs, exactly one level of nesting is required so that each key-value pair is contained within a section. The top level keys become sections, which are contained within square brackets `[]`. Read more about INI configuration files <a href="https://en.wikipedia.org/wiki/INI_file">here</a>.
<!--cell 14-->

In [8]:
config.get_ini_config(
    config={"message": {"greeting":"Salutations", "recipient":"Mars"}}
)

[message]
greeting = Salutations
recipient = Mars

Either more or fewer levels of nesting will raise a `UWConfigError`.
<!--cell 16-->

In [31]:
try:
    config.get_ini_config(
        config={"greeting":"Salutations", "recipient":"Mars"}
    )
except Exception as e:
    print(e)

Cannot instantiate depth-2 INIConfig with depth-1 config


## Realizing Configs

The `config.realize()` function writes config files to disk or `stdout` with the ability to render Jinja2 expressions and add/update values.
<!--cell 18-->

In [10]:
help(config.realize)

Help on function realize in module uwtools.api.config:

realize(input_config: Union[uwtools.config.formats.base.Config, pathlib.Path, dict, str, NoneType] = None, input_format: Optional[str] = None, update_config: Union[uwtools.config.formats.base.Config, pathlib.Path, dict, str, NoneType] = None, update_format: Optional[str] = None, output_file: Union[str, pathlib.Path, NoneType] = None, output_format: Optional[str] = None, key_path: Optional[list[Union[str, int]]] = None, values_needed: bool = False, total: bool = False, dry_run: bool = False, stdin_ok: bool = False) -> dict
    Realize a config based on a base input config and an optional update config.

    The input config may be specified as a filesystem path, a ``dict``, or a ``Config`` object. When it
    is not, it will be read from ``stdin``.

    If an update config is specified, it is merged onto the input config, augmenting or overriding base
    values. It may be specified as a filesystem path, a ``dict``, or a ``Config``

The `input_config` parameter takes a config from a string path, <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object, Python `dict`, or UW `Config` object like the `YAMLConfig` object from the <a href="#Getting-Config-Objects">Getting Config Objects</a> section. The `input_format` argument must be provided for `dict` inputs or for files without recognized extensions. Configs are written to `stdout` if `output_file` is unspecified or explicitly set to `None`, or to the file specified by `output_file`. The `output_format` argument must be provided when writing to `stdout` or to a file without a recognized extension. Recognized extensions are: `.ini`, `.nml`, `.sh`, and `.yaml`.
<!--cell 20-->

In [11]:
config.realize(
    input_config=config1,
    output_file=Path('tmp/config1.yaml')
)

{'greeting': 'Hello', 'recipient': 'World'}

The `realize()` method returns a dict version of the config regardless of input type, and the file is written in the YAML format as indicated by the file extension.
<!--cell 22-->

In [12]:
%%bash
cat tmp/config1.yaml

greeting: Hello
recipient: World


Input and output formats are not required to match. This can be used to convert a config from one format to another.
<!--cell 24-->

In [13]:
config.realize(
    input_config='fixtures/config/get-config.yaml',
    input_format='yaml',
    output_file='tmp/realize-config.sh',
    output_format='sh'
)

{'greeting': 'Hello', 'recipient': 'World'}

Here a Shell config is created from a YAML config.
<!--cell 26-->

In [14]:
%%bash
cat tmp/realize-config.sh

greeting=Hello
recipient=World


### Updating Configs

Configs can be updated by providing a second config with the `update_config` parameter. If the update config contains keys that match the base config, the base config values for those keys will be overwritten. Once updated, if the config contains Jinja2 expressions, like the one below, they will be rendered in the config wherever possible.
<!--cell 28-->

In [15]:
%%bash
cat fixtures/config/base-config.nml

&memo
  sender_id = "{{ id }}"
  message = "{{ greeting }}, {{ recipient }}!"
  sent = .FALSE.
/


Here, the update config provides values that will update two of the Jinja2 expressions and override one key with a new value.
<!--cell 30-->

In [16]:
config.realize(
    input_config='fixtures/config/base-config.nml',
    update_config={"memo": {"greeting":"Salutations", "recipient":"Mars", "sent": True}},
    output_file='tmp/updated-config.nml'
)

{'memo': {'sender_id': '{{ id }}',
  'message': 'Salutations, Mars!',
  'sent': True,
  'greeting': 'Salutations',
  'recipient': 'Mars'}}

All of the key-value pairs were added to the updated config, and the base config was rendered where the appropriate values were provided. However, not all Jinja2 expressions are required to be rendered: An `id` key was not provided in the update config, so the expression referencing it was not rendered.
<!--cell 32-->

In [17]:
%%bash
cat tmp/updated-config.nml

&memo
    sender_id = '{{ id }}'
    message = 'Salutations, Mars!'
    sent = .true.
    greeting = 'Salutations'
    recipient = 'Mars'
/


### Using the `key_path` Parameter

Consider the following config file, where the desired keys and values are not at the top level.
<!--cell 34-->

In [18]:
%%bash
cat fixtures/config/keys-config.yaml

keys:
  to:
    config:
      message: "{{ greeting }}, {{ recipient }}!"


The `key_path` parameter allows only a portion of the config, identified by following a given list of keys, to be written to a file or, in this case, to `stdout`. Note that the key-value pairs from the update config are used to render values and appear in the returned config dictionary, but don't appear in the config written to `stdout`.
<!--cell 36-->

In [19]:
config.realize(
    input_config="fixtures/config/keys-config.yaml",
    update_config={"greeting": "Good morning", "recipient": "Venus"},
    output_file=None,
    output_format='yaml',
    key_path=['keys', 'to', 'config']
)

message: Good morning, Venus!


{'keys': {'to': {'config': {'message': 'Good morning, Venus!'}}},
 'greeting': 'Good morning',
 'recipient': 'Venus'}

### Using the `values_needed` Parameter

Consider the config file below, which contains unrendered Jinja2 expressions.
<!--cell 38-->

In [20]:
%%bash
cat fixtures/config/base-config.nml

&memo
  sender_id = "{{ id }}"
  message = "{{ greeting }}, {{ recipient }}!"
  sent = .FALSE.
/


Setting `values_needed` to `True` will allow logging of keys that contain unrendered Jinja2 expressions and their values. A logger needs to be initialized for this information to be displayed. The config is not written and the returned `dict` is empty.
<!--cell 40-->

In [21]:
config.realize(
    input_config='fixtures/config/base-config.nml',
    output_file=None,
    output_format='nml',
    values_needed=True
)

[2024-09-23T13:14:27]     INFO Keys that are complete:
[2024-09-23T13:14:27]     INFO   memo
[2024-09-23T13:14:27]     INFO   memo.sent
[2024-09-23T13:14:27]     INFO 
[2024-09-23T13:14:27]     INFO Keys with unrendered Jinja2 variables/expressions:
[2024-09-23T13:14:27]     INFO   memo.sender_id: {{ id }}
[2024-09-23T13:14:27]     INFO   memo.message: {{ greeting }}, {{ recipient }}!


{}

### Using the `total` Parameter

The `total` parameter is used to specify that all Jinja2 expressions must be rendered before the final config is written. Consider the config below which contains multiple expressions.
<!--cell 42-->

In [22]:
%%bash
cat fixtures/config/base-config.nml

&memo
  sender_id = "{{ id }}"
  message = "{{ greeting }}, {{ recipient }}!"
  sent = .FALSE.
/


As was shown in the <a href="#Updating-Configs">Updating Configs</a> section, not all Jinja2 expressions are required to be rendered when `total` is set to its default value, `False`. However, a complete rendering of all expressions will be required if `total` is set to `True`. If this is the case and not enough values are provided to fully render the config, a `UWConfigRealizeError` will be raised. Notice that values are provided for `greeting` and `recipient`, but not for `id`.
<!--cell 44-->

In [32]:
try:
    config.realize(
        input_config='fixtures/config/base-config.nml',
        update_config={"memo": {"greeting":"Salutations", "recipient":"Mars", "sent":True}},
        output_file='tmp/config-total.nml',
        total=True
    )
except Exception as e:
    print(e)

Config could not be totally realized


With all values provided to fully render the config, `realize()` writes the complete config without error.
<!--cell 46-->

In [24]:
config.realize(
    input_config='fixtures/config/base-config.nml',
    update_config={"memo": {"greeting":"Salutations", "recipient":"Mars", "sent":True, "id":321}},
    output_file='tmp/config-total.nml',
    total=True
)

{'memo': {'sender_id': '321',
  'message': 'Salutations, Mars!',
  'sent': True,
  'greeting': 'Salutations',
  'recipient': 'Mars',
  'id': 321}}

The newly created config file is free from any unrendered Jinja2 expressions.
<!--cell 48-->

In [25]:
%%bash
cat tmp/config-total.nml

&memo
    sender_id = '321'
    message = 'Salutations, Mars!'
    sent = .true.
    greeting = 'Salutations'
    recipient = 'Mars'
    id = 321
/


## Realizing Configs to a Dictionary

The `config.realize_to_dict()` function has the ability to manipulate config values, and returns the config as a Python `dict`.
<!--cell 50-->

In [26]:
help(config.realize_to_dict)

Help on function realize_to_dict in module uwtools.api.config:

realize_to_dict(input_config: Union[uwtools.config.formats.base.Config, pathlib.Path, dict, str, NoneType] = None, input_format: Optional[str] = None, update_config: Union[uwtools.config.formats.base.Config, pathlib.Path, dict, str, NoneType] = None, update_format: Optional[str] = None, key_path: Optional[list[Union[str, int]]] = None, values_needed: bool = False, total: bool = False, dry_run: bool = False, stdin_ok: bool = False) -> dict
    Realize a config to a ``dict``, based on a base input config and an optional update config.

    See ``realize()`` for details on arguments, etc.



Like `realize()`, input or update configs can be Python dictionaries, UW `Config` objects, or files like the one below.
<!--cell 52-->

In [27]:
%%bash
cat fixtures/config/get-config.yaml

greeting: Hello
recipient: World


`realize_to_dict()` has the same parameters as `realize()`, with the exception of `output_file` and `output_format`. These parameters are not used because a config won't be written to a file or to `stdout` as `config.realize()` would. Instead, configs can be manipulated or converted to a `dict` without the need to specify an output file or format.
<!--cell 54-->

In [28]:
config.realize_to_dict(
    input_config={"id": "456"},
    update_config="fixtures/config/get-config.yaml"
)


{'id': '456', 'greeting': 'Hello', 'recipient': 'World'}

For more details on usage and parameters, see the <a href="#Realizing-Configs">Realizing Configs</a> section above.
<!--cell 56-->